In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,66105
2,Huelva,Confirmados PDIA 14 días,8140
3,Huelva,Tasa PDIA 14 días,"1586,2189917571175"
4,Huelva,Confirmados PDIA 7 días,3884
5,Huelva,Tasa PDIA 7 dias,"756,8641970497106"
6,Huelva,Total Confirmados,66326
7,Huelva,Curados,51894
8,Huelva,Fallecidos,431


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  66105.0


/tmp/ipykernel_52057/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  19359.0


/tmp/ipykernel_52057/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_52057/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_52057/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_52057/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 111 personas en los últimos 7 días 

Un positivo PCR cada 52 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,66105.0,8140.0,3884.0,513170.0,756.864197,1586.218992,558.0
Huelva-Costa,38777.0,4678.0,2220.0,289548.0,766.712255,1615.621590,261.0
Sierra de Huelva-Andévalo Central,7572.0,1564.0,755.0,67391.0,1120.327640,2320.784675,174.0
Huelva (capital),19359.0,2761.0,1296.0,143837.0,901.019904,1919.533917,154.0
Condado-Campiña,19294.0,1883.0,902.0,156231.0,577.350206,1205.266560,123.0
Cortegana,537.0,117.0,71.0,4602.0,1542.807475,2542.372881,42.0
Valverde del Camino,1644.0,292.0,125.0,12750.0,980.392157,2290.196078,27.0
Cartaya,2916.0,346.0,187.0,20083.0,931.135786,1722.850172,24.0
Aljaraque,2612.0,385.0,188.0,21474.0,875.477321,1792.865791,21.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,24.0,21.0,2.0,330.0,606.060606,6363.636364,0.0
Rosal de la Frontera,188.0,77.0,26.0,1697.0,1532.115498,4537.418975,9.0
Cerro de Andévalo (El),345.0,92.0,55.0,2327.0,2363.558229,3953.588311,11.0
Fuenteheridos,68.0,23.0,17.0,709.0,2397.743300,3244.005642,1.0
Villanueva de las Cruces,46.0,12.0,5.0,387.0,1291.989664,3100.775194,0.0
Zalamea la Real,276.0,93.0,44.0,3054.0,1440.733464,3045.186640,12.0
Campillo (El),198.0,61.0,25.0,2024.0,1235.177866,3013.833992,1.0
Jabugo,207.0,63.0,31.0,2260.0,1371.681416,2787.610619,6.0
Aroche,282.0,82.0,37.0,3054.0,1211.525868,2685.003274,10.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Hinojales,24.0,21.0,2.0,330.0,606.060606,6363.636364,0.0,0.095238
Santa Bárbara de Casa,64.0,11.0,2.0,1043.0,191.754554,1054.650048,0.0,0.181818
Encinasola,197.0,31.0,7.0,1310.0,534.351145,2366.412214,1.0,0.225806
Paterna del Campo,391.0,47.0,13.0,3457.0,376.048597,1359.560312,3.0,0.276596
Escacena del Campo,321.0,20.0,6.0,2287.0,262.352427,874.508089,1.0,0.300000
Campofrío,97.0,13.0,4.0,713.0,561.009818,1823.281907,0.0,0.307692
Villablanca,355.0,28.0,9.0,2885.0,311.958406,970.537262,3.0,0.321429
Santa Ana la Real,34.0,3.0,1.0,475.0,210.526316,631.578947,0.0,0.333333
Villanueva de los Castillejos,301.0,36.0,12.0,2825.0,424.778761,1274.336283,3.0,0.333333
